In [9]:
import os
import streamlit as st
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import SeleniumURLLoader, PyPDFLoader
from langchain.chains import RetrievalQA

load_dotenv()

api_key = os.getenv('GROQ_API_KEY')

In [10]:
def load_llm():
    # load the llm from Groq
    llm = ChatGroq(groq_api_key=api_key, model_name="llama3-70b-8192", temperature=0)
    return llm

In [11]:
def load_vector_store(urls):

    loader = SeleniumURLLoader(urls=urls)
    documents = loader.load()

    # load the vector store
    embeddings = HuggingFaceEmbeddings(
        model_name="thenlper/gte-large",
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
    )

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
    texts_chunks = text_splitter.split_documents(documents)

    db = Chroma.from_documents(texts_chunks, embeddings, persist_directory="db")
    return db

In [12]:
def load_prompt_template():

    # prepare the template we will use when prompting the AI
    template = """Use the provided context to answer the user's question.
    If you don't know the answer, respond with "I do not know".

    Context: {context}
    Question: {question}
    Answer:
    """

    prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'question'])

    return prompt 

In [14]:
def create_qa_chain():

    # load the llm, vector store, and the prompt
    llm = load_llm()
    db = load_vector_store(urls)
    prompt = load_prompt_template()

    # create the qa_chain
    retriever = db.as_retriever(search_kwargs={'k': 2})
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                        chain_type='stuff',
                                        retriever=retriever,
                                        return_source_documents=True,
                                        chain_type_kwargs={'prompt': prompt})
    
    return qa_chain

In [15]:
def generate_response(query, qa_chain):

    # use the qa_chain to answer the given query
    return qa_chain({'query':query})['result']

In [16]:
urls = [
    "https://incelligent.net/case-studies/",
    "https://incelligent.net/about-us/",
]

qa_chain = create_qa_chain()

In [17]:
response = generate_response(query="Who is Ignatios Chatzigianellis?", qa_chain=qa_chain)
print(response)

Ignatios Chatzigianellis is a Machine Learning Engineer at Incelligent.
